In [1]:
%load_ext autoreload

# (4) 测试去噪算法

- TianmoucDataReader 的測試功能
    - ## [输入]
    - aop_denoise, 是否開啓AOP去噪
    - aop_denoise_args = denoise_defualt_args(), 去噪默認參數

## 重要：去噪参数配置方法
```python
denoise_defualt_args(thr_1 = 2, 
                    thr_2 = 5, 
                    thr_3 = 5, 
                    aop_gain = 1, 
                    self_calibration=False)

- thr_1: 模板阈值
- thr_2: 3*3池化阈值
- thr_3: 空洞池化阈值
- aop_gain: 需要匹配不同AOP增益
- self_calibration: 如果没有暗噪声，是否用自己的aop标定暗噪声（对于亮环境不一定好）

# 对比去噪和不去噪的结果

In [ ]:
%autoreload
import torch
import matplotlib.pyplot as plt

from tianmoucv.data import TianmoucDataReader
from tianmoucv.isp import vizDiff
from tianmoucv.data.denoise_utils import denoise_defualt_args

aim = 4 # 读取的目标COP帧的帧号
N = 1   # read continue N frames （连续读取N帧COP以及对应的AOP）

datapath = '/data/lyh/tianmoucData/20240930_tobi_sup_exp/data/lowlight_highspeed/tianmouc/100lux/1515fps_opt' #灰阶图
key_list = ['1500rpm_g1']

for key in key_list:

    # 默认关闭去噪
    dataset = TianmoucDataReader(datapath,matchkey=key,aop_denoise = False,camera_idx=0)

    # 开启去噪
    #g4，所以用aop_gain=4
    new_aop_denoise_args= denoise_defualt_args(thr_1 = 2, 
                                               thr_2 = 5, 
                                               thr_3 = 5, 
                                               aop_gain = 1, 
                                               self_calibration=False)
    denoised_dataset = TianmoucDataReader(datapath,
                                          matchkey=key,
                                          aop_denoise = True, 
                                          aop_denoise_args = new_aop_denoise_args, 
                                          camera_idx=0)
    
    img_list = []
    for index in range(aim,min(aim+1,len(dataset))):
        sample = dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        print(sample['meta'])
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0

            tsd_rgb = tsdiff[:,i,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)
            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()

    print('去噪结果')
    for index in range(aim,min(aim+1,len(denoised_dataset))):
        sample = denoised_dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0
            tsd_rgb = tsdiff[:,i,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()
        

# 如果录制了暗电流噪声，可以使用完整版本

In [ ]:
%autoreload

from tianmoucv.isp import vizDiff
from tianmoucv.data.denoise_utils import denoise_defualt_args

# 用于生成需要的Dark Noise Array
dark_noise_path = '/data/lyh/tianmoucData/20240930_tobi_sup_exp/data/tianmouc_dark/dark_gain1'

def aop_denoise_arg_generator(dark_noise_path = None,aop_gain=1):
    new_aop_denoise_args= denoise_defualt_args(thr_1 = 1, 
                                               thr_2 = 3, 
                                               thr_3 = 3, 
                                               aop_gain = aop_gain, 
                                               self_calibration=False)
    dataset_dark = TianmoucDataReader(dark_noise_path,aop_denoise = True,aop_denoise_args = denoise_defualt_args(self_calibration=True))
    new_aop_denoise_args.aop_dark_dict = dataset_dark.get_dark_noise()
    return new_aop_denoise_args


for key in key_list:
    aop_gain = 1
    aop_denoise_args = aop_denoise_arg_generator(dark_noise_path,aop_gain)
    denoised_dataset = TianmoucDataReader(datapath,matchkey=key,
                                          aop_denoise = True, 
                                          aop_denoise_args = aop_denoise_args, 
                                          camera_idx=0)

    print('减去暗噪声的去噪结果')
    N = 1
    for index in range(aim,min(aim+5,len(denoised_dataset))):
        sample = denoised_dataset[index]
        tsdiff = torch.Tensor(sample['tsdiff'])
        length = tsdiff.shape[1]
        plt.figure(figsize=(12,2*N))  
        for i in range(N):
            F = sample['F'+str(i)] #第i帧COP
            F_HDR = sample['F'+str(i)+'_HDR'] #第i帧的HDR合成结果
            F_HDR[F_HDR>1]=1
            F_HDR[F_HDR<0]=0

            tsd_rgb = tsdiff[:,12,...].permute(1,2,0)*255
            td = tsd_rgb.cpu()[:,:,0]
            sd = tsd_rgb.cpu()[:,:,1:]
            rgb_sd = vizDiff(sd,thresh=2)
            rgb_td = vizDiff(td,thresh=2)
            rgb_cat = torch.cat([rgb_sd,rgb_td],dim=1).permute(1,2,0)
            
            plt.subplot(N,3,1+i*3)
            plt.imshow(F)
            plt.subplot(N,3,2+i*3)
            plt.imshow(rgb_cat) #可视化第0帧，乘个Gain方便看
            plt.subplot(N,3,3+i*3)
            plt.imshow(F_HDR)
        plt.show()
        